In [63]:
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
import librosa.display
import sklearn
import pandas as pd
from keras.models import model_from_json
import os

# Set the environments

In [3]:
%matplotlib inline

In [4]:
plt.style.use('fivethirtyeight')
%matplotlib inline
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# Load the Files

In [29]:
all_files=[]
directory = r'./data'
for filename in os.listdir(directory):
    dict={}
    dict['name']=filename
    dict['dir']=os.path.join(directory, filename)
    all_files.append(dict)
    pass

# Helper

In [31]:
def normalize(x, axis=0):
    return sklearn.preprocessing.minmax_scale(x, axis=axis)

# Prepare a training Dataset with Spectral Centroid

In [54]:
all_spectral_centroids=[]
filenamea=[]
for audio_file in all_files:
    filenamea.append(audio_file['name'])
    x,sr = librosa.load(audio_file['dir'], sr=44100)
    spectral_centroids = librosa.feature.spectral_centroid(x, sr=sr)[0]
    nx=normalize(spectral_centroids)
    if len(nx)==216:
        spectral_centroids_x = list(nx)
        all_spectral_centroids.append(spectral_centroids_x)
        
    else:
        print("Error in file length! LEN=",str(len(nx)),audio_file)

In [56]:
columns=[]
for i in range(216):
    columns.append("SC"+str(i))
df_sc = pd.DataFrame(columns=columns, data=all_spectral_centroids)

In [59]:
filesN=pd.DataFrame(columns=['NAME'], data=filenamea)

# Load Model

In [64]:
json_file = open('sound_model.v3.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("sound_model.v3.h5")
print("Loaded model from disk")

Loaded model from disk


# Prediction

In [66]:
y_pred = model.predict_classes(df_sc,verbose=0)

In [70]:
array_new_pred=[]
for y in y_pred:
    if(y==0):
        array_new_pred.append('0 deg')
        pass
    elif(y==1):
        array_new_pred.append('90 deg')
        pass
    else:
        array_new_pred.append('180 deg')
        pass
    pass

# Prepare the Report

In [71]:
y_pred_df=pd.DataFrame(columns=['Angles'], data=array_new_pred)

In [73]:
report_df=pd.concat([filesN, y_pred_df], axis=1)

In [74]:
report_df

,NAME,Angles
0,file_4.wav,0 deg
1,file_5.wav,90 deg
2,file_2.wav,90 deg
3,file_3.wav,180 deg
4,file_1.wav,0 deg
